In [1]:
import os 

In [2]:
os.chdir("../")

In [3]:
!pwd

/Users/sanjaymahto/Chicken-Disease-Classifiaction


In [5]:
from dataclasses import dataclass
from pathlib import Path 



In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_path: Path
    unzip_dir: Path

What i need to do now I need to read the yaml file so read the yaml file what do you need to do you need to first of all mention this thing in the constant so, i'll open the constant folder 

In [13]:

from src.constants import * 
from src.utils.common import read_yaml, create_directories
import config

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir), 
            source_url=config.source_URL,
            local_data_path=Path(config.local_data_file), 
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [15]:
import os 
import urllib.request as request
import zipfile
from src import logger
from src.utils.common import get_size


In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_path):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename=self.config.local_data_path
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_path))}")
            
    def extract_zip_file(self):
        """Extracts the zip file into the data directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
# Example usage
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error: {e}")
    raise e

[2024-08-05 11:04:00,404: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/config/config.yaml loaded successfully]
[2024-08-05 11:04:00,409: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/params.yaml loaded successfully]
[2024-08-05 11:04:00,412: INFO: common: created directory at: artifacts]
[2024-08-05 11:04:00,415: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-05 11:04:28,349: INFO: 258208126: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6705:17E462:1C5AE3:273D1D:66B0644C
Accept-Rang